# Installation of Libraries

In [1]:
!pip install pyyaml==5.4.1
!pip install gensim
!pip install sentence-transformers
!pip install tensorflow-hub
!pip install transformers
!pip install spacymoji
!pip install spacy==3.0.5
!pip install ekphrasis
!pip install preprocessor
!pip install ttp

     |████████████████████████████████| 636 kB 8.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 4.0 MB/s 
     |████████████████████████████████| 4.0 MB 16.3 MB/s 
     |████████████████████████████████| 1.2 MB 12.2 MB/s 
     |████████████████████████████████| 77 kB 5.6 MB/s 
     |████████████████████████████████| 895 kB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 33.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=24f3ce97939dc66c5b5c850a9e0ed6e516b1071489b7875af49aea3a213a6998
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
     |████████████████████████████████| 6.0 MB 4.4 MB/s 
     |████████████████████████████████| 51 kB 6.6 MB/s 
     |███

In [2]:
import spacy
import spacymoji
import re
import torch
import pickle
import bz2
import nltk
import torch
import random
import gensim
import itertools
import math
import json
import csv
import urllib.request
import datetime
import emoji
import ekphrasis
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import torch.nn as nn
import gensim.downloader as api
import torch.nn.functional as F
import tensorflow_hub as hub
import preprocessor as tweet_preprocessor
import sklearn

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, scale
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline
from os.path import join as pjoin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from matplotlib.pyplot import figure
from collections import Counter
from scipy import spatial
from scipy.special import softmax
from torch.utils.data import TensorDataset, DataLoader
from sentence_transformers import SentenceTransformer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.corpus import stopwords
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoTokenizer
from dateutil.parser import parse
from ttp import ttp
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [3]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
base_dir = 'drive/MyDrive/CASCS505_FinalProj'

# Load datasets

In [5]:
train = pd.read_json(pjoin(base_dir, 'train.zip'))
test = pd.read_json(pjoin(base_dir, 'test_questions.zip'))

In [6]:
# full_data = pd.read_pickle('data.pkl')

In [6]:
files.upload()

Saving test_vader_sentiment.csv to test_vader_sentiment.csv
Saving train_vader_sentiment.csv to train_vader_sentiment.csv
Saving test_ids_glove.csv to test_ids_glove.csv
Saving train_ids_glove.csv to train_ids_glove.csv


In [ ]:
train_vader = pd.read_csv('train_vader_sentiment.csv')
test_vader = pd.read_csv('test_vader_sentiment.csv')
train_glove = pd.read_csv('train_ids_glove.csv', header=None)
test_glove = pd.read_csv('test_ids_glove.csv', header=None)

In [ ]:
train_vader.columns = ['id', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']
test_vader.columns = ['id', 'vader_neg', 'vader_neu', 'vader_pos', 'vader_compound']

In [ ]:
train = train.set_index('id').join(train_vader.set_index('id'))
test = test.set_index('id').join(test_vader.set_index('id'))

In [ ]:
glove_train_val = train_glove.set_index(list(train_glove.columns[[0]])).values.tolist()
train_glove_lst = []
for i in glove_train_val:
  train_glove_lst.append(i)

glove_test_val = test_glove.set_index(list(test_glove.columns[[0]])).values.tolist()
test_glove_lst = []
for i in glove_test_val:
  test_glove_lst.append(i)

In [20]:
train_glove = train_glove.set_index(list(train_glove.columns[[0]]))
test_glove = test_glove.set_index(list(test_glove.columns[[0]]))

In [27]:
train_glove_modified = pd.DataFrame()
train_glove_modified['id'] = train_glove.index
train_glove_modified['glove'] = train_glove_lst

In [29]:
test_glove_modified = pd.DataFrame()
test_glove_modified['id'] = test_glove.index
test_glove_modified['glove'] = test_glove_lst

In [36]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [37]:
train = train.set_index('id').join(train_glove_modified.set_index('id'))
test = test.set_index('id').join(test_glove_modified.set_index('id'))

In [38]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

# Pre Processing

In [39]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

def preprocess_tweets(dataframe, text_processor):
  processed_lst = dataframe['text'].str.lower()
  temp = []
  for s in processed_lst:
    s = (" ".join(text_processor.pre_process_doc(s)))
    s = re.sub(r"[^\w\d\s\<\>]+", '', s)
    temp.append(s)
  return temp

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [40]:
train['processed'] = preprocess_tweets(train, text_processor = text_processor)
test['processed'] = preprocess_tweets(test, text_processor = text_processor)

In [41]:
stop_words = stopwords.words('english')
train['processed'] = train['processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
train['processed'] = train['processed'].replace('_', '', regex=True)
test['processed'] = test['processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
test['processed'] = test['processed'].replace('_', '', regex=True)

In [42]:
tt = TweetTokenizer()
train['tokenized'] = train['processed'].apply(tt.tokenize)
test['tokenized'] = test['processed'].apply(tt.tokenize)

In [43]:
train['type'] = 'train'
test['type'] = 'test'

In [44]:
#intersect = np.intersect1d(train.columns, test.columns)
#train = train[intersect]
#test = test[intersect]

In [45]:
full_data = pd.concat([train, test], ignore_index=True, sort=False)

# Sentence Embeddings

## Glove

## TFIDF

In [49]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vector = tfidf_vectorizer.fit_transform(full_data.processed)

In [58]:
tfidf_lst = []
train_lst_tfidf = tfidf_vector[:len(train)] 
for idx, i in enumerate(train_lst_tfidf):
  tfidf_lst.append(train_lst_tfidf[idx])

train['tfidf'] = tfidf_lst

In [62]:
tfidf_lst = []
test_lst_tfidf = tfidf_vector[len(train):] 
for idx, i in enumerate(test_lst_tfidf):
  tfidf_lst.append(test_lst_tfidf[idx])

test['tfidf'] = tfidf_lst

## Count Vectorizer

In [63]:
count_vectorizer = CountVectorizer()
cv_vector = count_vectorizer.fit_transform(full_data.processed)

In [65]:
cv_lst = []
train_lst_cv = cv_vector[:len(train)] 
for idx, i in enumerate(train_lst_cv):
  cv_lst.append(train_lst_cv[idx])

train['cv'] = cv_lst

In [66]:
cv_lst = []
test_lst_cv = cv_vector[len(train):] 
for idx, i in enumerate(test_lst_cv):
  cv_lst.append(test_lst_cv[idx])

test['cv'] = cv_lst

## Bert

In [67]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [68]:
sbert_sentence_embeddings = []
for sentence in tqdm(train['processed'].tolist(), total = len(train)):
  sbert_sentence_embeddings.append(sbert_model.encode(sentence))

train['bert'] = sbert_sentence_embeddings

100%|██████████| 96562/96562 [3:54:34<00:00,  6.86it/s]


In [69]:
sbert_sentence_embeddings = []
for sentence in tqdm(test['processed'].tolist(), total = len(test)):
  sbert_sentence_embeddings.append(sbert_model.encode(sentence))

test['bert'] = sbert_sentence_embeddings

100%|██████████| 32210/32210 [1:17:18<00:00,  6.94it/s]


## Doc2Vec

In [70]:
tokenized_sent = []
for s in tqdm(full_data['processed'].tolist(), total = len(full_data)):
  tokenized_sent.append(word_tokenize(s.lower()))

100%|██████████| 128772/128772 [00:22<00:00, 5818.38it/s]


In [71]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_sent)]

In [72]:
d2v_model = Doc2Vec(tagged_data, vector_size = 100, window = 2, min_count = 1, epochs = 100)

In [73]:
d2v_sentence_emebeddings = []
for sentence in tqdm(train['processed'].tolist(), total = len(train)):
  doc = word_tokenize(sentence.lower())
  d2v_sentence_emebeddings.append(d2v_model.infer_vector(doc))

train['d2v'] = d2v_sentence_emebeddings

100%|██████████| 96562/96562 [08:42<00:00, 184.94it/s]


In [74]:
d2v_sentence_emebeddings = []
for sentence in tqdm(test['processed'].tolist(), total = len(test)):
  doc = word_tokenize(sentence.lower())
  d2v_sentence_emebeddings.append(d2v_model.infer_vector(doc))

test['d2v'] = d2v_sentence_emebeddings

100%|██████████| 32210/32210 [02:54<00:00, 184.81it/s]


## Universal Sentence Encoder

In [75]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
use_model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [76]:
use_sentence_embeddings = []
for sentence in tqdm(train['processed'].tolist(), total = len(train)):
  use_sentence_embeddings.append(use_model([sentence]))

train['use'] = use_sentence_embeddings

100%|██████████| 96562/96562 [06:58<00:00, 230.62it/s]


In [77]:
use_sentence_embeddings = []
for sentence in tqdm(test['processed'].tolist(), total = len(test)):
  use_sentence_embeddings.append(use_model([sentence]))

test['use'] = use_sentence_embeddings

100%|██████████| 32210/32210 [02:16<00:00, 235.56it/s]


# Sentiment

## Roberta

In [ ]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
labels=[]

mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
  html = f.read().decode('utf-8').split("\n")
  csvreader = csv.reader(html, delimiter='\t')

labels = [row[1] for row in csvreader if len(row) > 1]

In [ ]:
auto_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
auto_model.save_pretrained(MODEL)

In [ ]:
auto_scores_lst = []
#tf_scores_lst = []
for sentence in tqdm(full_data['processed'].tolist(), total = len(full_data)):
  auto_encoded_input = tokenizer(sentence, return_tensors='pt')
  auto_output = auto_model(**auto_encoded_input)
  auto_scores = auto_output[0][0].detach().numpy()
  auto_scores = softmax(auto_scores)

  auto_scores_lst.append(auto_scores)

In [ ]:
full_data['auto_sentiment'] = auto_scores_lst

## Vader

# Feature Extraction

## Train

In [78]:
custom_columns=["tweet_id", "timestamp", "year", "month", "week_day", "hour", "is_reply_to_tweet", "is_reply_to_user", "user_id", "@_count", "#_count", "url_count", "tweet_len", "rewteet_count", "favorite_count"]
features = []
for idx, twt in train.iterrows():
  row = [twt["id_str"]]
  time = twt['created_at']
  row.extend([int(time.timestamp()), time.year, time.month, time.weekday(), time.hour])
  if twt["in_reply_to_status_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  if twt["in_reply_to_user_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  row.append(twt["user"]["id_str"])
  row.append(len(twt["entities"]["user_mentions"]))
  row.append(len(twt["entities"]["hashtags"]))
  row.append(len(twt["entities"]["urls"]))
  row.append(len(twt["text"]))
  row.append(twt["retweet_count"])
  row.append(twt["favorite_count"])

  features.append(row)

In [79]:
feature_df = pd.DataFrame(data = features, columns = custom_columns)

In [80]:
aggregate_columns=["tweet_id", "user_total_retweeted_so_far", "user_total_favorited_so_far", "user_total_retweeted", "user_total_favorited"]
aggregate_features=[]
for user_id, group in feature_df.groupby('user_id'):
  group_row = []
  total_retweet = 0
  total_favorite = 0
  for twt in group.iterrows():
    row = [twt[1]["tweet_id"]]
    total_retweet += twt[1]["rewteet_count"]
    total_favorite += twt[1]["favorite_count"]
    row.extend([total_retweet, total_favorite])
    group_row.append(row)
  for i in range(len(group_row)):
    group_row[i].extend([total_retweet, total_favorite])
  aggregate_features.extend(group_row)

In [81]:
agg_df = pd.DataFrame(data = aggregate_features, columns = aggregate_columns)

In [82]:
custom_features_df = pd.merge(feature_df, agg_df, on = "tweet_id")

In [83]:
custom_features_df.reset_index(inplace=True, drop=True)
full_data.reset_index(inplace=True, drop=True)

In [84]:
train = pd.concat([train, custom_features_df], axis = 1)

## Test

In [85]:
custom_columns=["tweet_id", "timestamp", "year", "month", "week_day", "hour", "is_reply_to_tweet", "is_reply_to_user", "user_id", "@_count", "#_count", "url_count", "tweet_len"]
features = []
for idx, twt in test.iterrows():
  row = [twt["id_str"]]
  time = twt['created_at']
  row.extend([int(time.timestamp()), time.year, time.month, time.weekday(), time.hour])
  if twt["in_reply_to_status_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  if twt["in_reply_to_user_id_str"]==None:
    row.append(0)
  else:
    row.append(1)

  row.append(twt["user"]["id_str"])
  row.append(len(twt["entities"]["user_mentions"]))
  row.append(len(twt["entities"]["hashtags"]))
  row.append(len(twt["entities"]["urls"]))
  row.append(len(twt["text"]))

  features.append(row)

In [86]:
feature_df = pd.DataFrame(data = features, columns = custom_columns)

In [87]:
feature_df.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [88]:
test = pd.concat([full_data, feature_df], axis = 1)

# Save Data as pickle

In [89]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [ ]:
files.download('train.pkl')

In [ ]:
files.download('test_pkl')

# Linear Regression Model

## Glove

## TFIDF

## Count Vectorizer

## Bert

## Doc2Vec

## Universal Sentence Embeddings

In [ ]:
linear_regression = LinearRegression()  
linear_regression.fit(x_train, y_train)